In [5]:
import pandas as pd
from nltk import ConditionalFreqDist, bigrams, trigrams
from sklearn.feature_extraction.text import TfidfVectorizer
from typing_assistant.indexing import Collection, InvertedIndex, Lexicon

In [6]:
df = pd.read_csv('../data/captions_0.1.tsv', sep='\t', index_col='id')[: 10000]
corpus = df['caption'].to_dict()
len(corpus)

10000

In [8]:
collection = Collection()
collection.build_collection(corpus)
inv_index = InvertedIndex(collection)
inv_index.index_collection()
lexicon = Lexicon()
lexicon.build_lexicon(collection, inv_index)

In [11]:
vectorizer = TfidfVectorizer(token_pattern=r'\w+')
tfidf_matrix = vectorizer.fit_transform([sentence for sentence in corpus.values()])
features = vectorizer.get_feature_names()
sums = tfidf_matrix.sum(axis=0)
sorted({term: sums[0, col] for col, term in enumerate(features)}.items(), key=lambda x: x[1], reverse=True)[: 10]

[('the', 624.2778225105985),
 ('a', 531.1542926055173),
 ('of', 394.1603911971106),
 ('on', 325.6256597061032),
 ('in', 311.4396563250166),
 ('and', 277.02504262038576),
 ('person', 239.60072866678902),
 ('with', 236.04490675312425),
 ('at', 194.4727102813845),
 ('during', 192.05227336830671)]

In [ ]:
words =  (word for words_sentence in words_sentences for word in words_sentence)
bgs = tuple(bigrams(words))